# 벡터화 (Vectorization)

- 머신러닝 모델은 기계이기 때문에, 텍스트보다는 수치화된 숫자를 더 잘 이해함
- 자연어를 더 잘 이해하게 하기 위해 전처리 과정에서 텍스트를 숫자 벡터로 변환하는 벡터화 진행
- 통계와 머신 러닝을 활용한 방법과 인공 신경망을 활용하는 방법이 존재
- 이번 챕터에서는 통계와 머신 러닝을 활용한 방법을 알아볼 예정

## Cosine Similarity

- 두 벡터 사이의 코사인 각도를 이용하여 유사도를 구하는 방법
- 벡터의 방향 180도라면 유사도가 -1
- 벡터가 90도로 직각이라면 유사도 0
- 벡터의 방향이 완전하게 동일하다면 1
- [참고 자료](https://wikidocs.net/24603)

<img src='./imgs/cosS.png' width = 450>

<img src='./imgs/cosEx.png' width = "350" height = "250">

In [6]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :',cos_sim(doc2, doc3))


문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


## BOW (Bag of Words)

- 단어들의 가방
- Key는 단어, Value는 각 단어의 등장 횟수
- 단, 단어장(Vocabulary)는 등장하는 단어의 갯수를 의미! BOW와 다름

In [1]:
# doc1 = 'John likes to watch movies. Mary likes movies too.'
# BoW1 = {"John":1, "likes":2, "to":1, "watch":1, "movies":2, "Mary":1, "too":1}

### Keras

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow에 저장

print("Bag of Words :", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


### Sklearn

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.
print('단어장(Vocabulary)의 크기 :', len(vector.vocabulary_))

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}
단어장(Vocabulary)의 크기 : 10


### DTM(Document-Term Matrix)

- 문서 - 단어 행렬
- 여러 문서의 Bag of Words를 하나의 행렬로 구현한 것
- 즉, 각 문서에 등장한 단어의 빈도수를 하나의 행렬로 통합
- DTM은 **문서를 행**으로, 단어를 열로 가지는 행렬
- 반대로 TDM은 **단어를 행**으로, 문서를 열로 가지는 행렬

<img src='./imgs/dtm.png'>

- 위의 예시에서 행이 문서벡터, 열이 단어벡터
- 문서가 많아질수록 대부분의 값이 0이 되는 성질이 존재

<img src='./imgs/dtm2.png' width = 350>

In [7]:
# 위 얘시를 벡터로 표현하여 각 문서 벡터의 유사도를 구해보자

import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [8]:
print('{:.2f}'.format(cos_sim(doc1, doc2))) #문서1과 문서2의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc1, doc3))) #문서1과 문서3의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc2, doc3))) #문서2과 문서3의 코사인 유사도

0.67
0.67
1.00


#### DTM의 구현과 한계

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',    
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


**한계**
- 문서의 수와 단어의 수가 계속 늘어날수록, 행과 열은 대부분의 값이 0을 가진다는 특징
- 즉슨, 차원의 저주를 야기 & 저장 공간 낭비
- 단어의 빈도에만 집중하는 방법 자체의 한계
- 비교하는 2개의 문서에 'the'라는 문장이 많다고 두 문서는 비슷할까?

## TF-IDF

- Term Frequency-Inverse Document Frequency
- 단어 빈도 - 역문서 빈도
- 단어의 빈도와 문서의 빈도의 역수를 활용
- 'the'같이 불용어처럼 중요도가 낮으면서 모든 문서에 등장하는 단어들이 노이즈가 되는 것을 완화해줌
- 하지만 언제나 DTM보다 성능이 좋지는 않다.
- DTM을 만든 뒤, TF-IDF 가중치를 DTM에 적용
---

<img src="./imgs/TF-IDF.png" width = 350>

**수식 설명**
- x가 단어, y를 문서라고 가정
- TF는 각 문서에 등장하는 단어의 빈도
- 뒤쪽 log항이 IDF
- df = 문서 빈도, N = 전체 문서의 수

예시
- 전체 문서의 수가 5개라고 해봅시다. 그리고 단어 'like'가 문서2에서 200번, 문서 3에서 300번 등장했다고 해봅시다. 다른 문서에서 단어 'like'는 등장하지 않았습니다. 이때, 단어 'like'의 IDF는 몇일까요?
- like가 등장한 문서는 총 2개, 전체 문서의 수는 N
- 즉, df = 2, N= 5
- IDF = log(2/5)
---
문서2에서 'like'의 TF-IDF
- 등장 횟수(TF) x IDF = 200  x IDF
- = 200 x log(2/5)

문서3에서 'like'의 TF-IDF
- 등장 횟수 (TF) = 300
- = 300 x log(2/5)

### TF-IDF 구현

In [10]:
from math import log
import pandas as pd
print('완료')

완료


In [11]:
docs = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [12]:
N = len(docs) # 총 문서의 수
N

3

In [13]:
def tf(t, d):
    return d.count(t)
 
def idf(t):
    df = 0
    for doc in docs:
        df += t in doc    
    return log(N/(df + 1)) + 1
 
def tfidf(t, d):
    return tf(t,d)* idf(t)

In [14]:
## TF

result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tf(t, d))
        
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [15]:
## IDF

result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns=["IDF"])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


In [16]:
## TF - IDF

result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


### TF-IDF _ Scikit-learn

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(tfidfv.vocabulary_.keys()) # 단어장을 리스트로 저장
vocab.sort() # 단어장을 알파벳 순으로 정렬

# TF-IDF 행렬에 단어장을 데이터프레임의 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns = vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


## LSA & LDA

- [차원의 저주]("https://y-halfmoon.tistory.com/entry/Dimension")
- BOW를 기반으로 한 DTM과 TF-IDF는 단어의 의미를 벡터로 표현하지 못함

### LSA 실습

#### 필요 도구 import와 dataset download

In [18]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [19]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/sanghyun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanghyun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sanghyun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import os

csv_filename = './data/abcnews-date-text.csv'

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename=csv_filename)

('./data/abcnews-date-text.csv', <http.client.HTTPMessage at 0x7fab82353b20>)

In [21]:
data = pd.read_csv(csv_filename, on_bad_lines='skip')
data.shape

(1082168, 2)

In [22]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [23]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [24]:
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

headline_text    1054983
dtype: int64

In [25]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

#### 데이터 정제 및 정규화

In [26]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [27]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


#### 역토큰화 및 DTM 생성

- DTM을 생성하는 CountVectorizer 또는 TF-IDF 행렬을 생성하는 TfidfVectorizer의 입력으로 사용하기 위해서 토큰화 과정을 역으로 되돌리는 역토큰화

In [28]:
# 역토큰화 (토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)

train_data = detokenized_doc
print('=3')

=3


In [29]:
# 샘플 확인

train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

In [30]:
# CountVectorizer를 사용하여 DTM을 생성. 단어의 수는 5,000개로 제한
# 상위 5000개의 단어만 사용
c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)
print('=3')

=3


In [31]:
# DTM의 크기(shape)는 (문서의 수 × 단어 집합의 크기)

print('행렬의 크기 :',document_term_matrix.shape)

행렬의 크기 : (1054983, 5000)


#### TruncatedSVD _ Scikit-learn

In [33]:
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 1.20379244e-02, -3.69666569e-03,  1.82823632e-02, ...,
         4.34824551e-03, -2.51750364e-05,  1.26328298e-02],
       [ 2.90664009e-02, -1.08186499e-02,  1.83609472e-02, ...,
         5.66047941e-04, -9.37140341e-03, -6.43439400e-03],
       [ 5.03461221e-03, -2.01762514e-03,  9.78910181e-03, ...,
        -1.78175912e-03,  2.05478074e-03,  1.87218834e-03],
       ...,
       [ 2.97122218e-02,  4.16429655e-03,  2.51456460e-02, ...,
         3.35858326e-02,  4.62425327e-03,  1.70188331e-02],
       [ 6.17066854e-02, -6.43948988e-03,  1.36294009e-01, ...,
         9.78546992e-01,  7.35713552e-01, -3.67972895e-01],
       [ 7.12573136e-02,  2.83149618e-02, -1.11142897e-03, ...,
         1.44690659e-02, -5.46368168e-03,  1.83311567e-02]])

In [34]:
print(lsa_model.components_.shape)

(10, 5000)


In [35]:
terms = c_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74639), ('man', 0.45356), ('charge', 0.21096), ('new', 0.14089), ('court', 0.11149)]
Topic 2: [('man', 0.69417), ('charge', 0.30005), ('court', 0.16783), ('face', 0.11424), ('murder', 0.10699)]
Topic 3: [('new', 0.83645), ('plan', 0.23599), ('say', 0.18269), ('govt', 0.11048), ('council', 0.1094)]
Topic 4: [('say', 0.73831), ('plan', 0.3599), ('govt', 0.16721), ('council', 0.13192), ('fund', 0.07439)]
Topic 5: [('plan', 0.73094), ('council', 0.1707), ('govt', 0.14146), ('urge', 0.08653), ('water', 0.07566)]
Topic 6: [('govt', 0.53944), ('court', 0.25769), ('urge', 0.24312), ('fund', 0.21304), ('face', 0.16885)]
Topic 7: [('charge', 0.52207), ('court', 0.46823), ('face', 0.3317), ('plan', 0.12449), ('murder', 0.12389)]
Topic 8: [('win', 0.65772), ('court', 0.29973), ('kill', 0.20791), ('crash', 0.14231), ('australia', 0.10563)]
Topic 9: [('win', 0.55482), ('charge', 0.51754), ('open', 0.06653), ('council', 0.06407), ('cup', 0.0635)]
Topic 10: [('council', 0.67847)

### LDA 실습

In [36]:
# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


In [37]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.17024867, 0.0335099 ,
        0.0335099 ],
       [0.03365631, 0.03365631, 0.03365631, ..., 0.03365631, 0.03365631,
        0.03365631],
       [0.25184095, 0.0366096 , 0.0366096 , ..., 0.0366096 , 0.0366096 ,
        0.0366096 ],
       ...,
       [0.26687206, 0.02914502, 0.02914502, ..., 0.13007484, 0.02916018,
        0.28739608],
       [0.10378115, 0.02637829, 0.12325014, ..., 0.02637829, 0.02637829,
        0.02637829],
       [0.03376055, 0.03376055, 0.2255442 , ..., 0.03376055, 0.03376055,
        0.03376055]])

In [38]:
print(lda_model.components_.shape)

(10, 5000)


In [39]:
# LDA의 결과 토픽과 각 단어의 비중을 출력합시다
terms = tfidf_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])

get_topics(lda_model.components_, terms)

Topic 1: [('australia', 9359.06334), ('sydney', 5854.97288), ('attack', 4784.76322), ('change', 4193.63035), ('year', 3924.88997)]
Topic 2: [('government', 6344.07413), ('charge', 5947.12292), ('man', 4519.7974), ('state', 3658.16422), ('live', 3625.10473)]
Topic 3: [('australian', 7666.65651), ('say', 7561.01807), ('police', 5513.22932), ('home', 4048.38409), ('report', 3796.04446)]
Topic 4: [('melbourne', 5298.35047), ('south', 4844.59835), ('death', 4281.78433), ('china', 3214.44581), ('women', 3029.28443)]
Topic 5: [('win', 5704.0914), ('canberra', 4322.0963), ('die', 4025.63057), ('open', 3771.65243), ('warn', 3577.47151)]
Topic 6: [('court', 5246.3124), ('world', 4536.86331), ('country', 4166.34794), ('woman', 3983.97748), ('crash', 3793.50267)]
Topic 7: [('election', 5418.5038), ('adelaide', 4864.95604), ('house', 4478.6135), ('school', 3966.82676), ('2016', 3955.11155)]
Topic 8: [('trump', 8189.58575), ('new', 6625.2724), ('north', 3705.40987), ('rural', 3521.42659), ('donald',

### 형태소 분석기의 필요성

In [40]:
en_text = "The dog ran back to the corner near the spare bedrooms"
print(en_text.split())

['The', 'dog', 'ran', 'back', 'to', 'the', 'corner', 'near', 'the', 'spare', 'bedrooms']


In [41]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사', '왔어']


In [42]:
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


In [43]:
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워'))

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


#### soynlp

- 비지도 학습으로 형태소 분석을 한다는 특징
- 데이터에 자주 등장하는 단어들을 형태로소 분석
- 내부적으로 '단어 점수표'에 의해 동작.
- 이 점수는 응집 확률과 브랜칭 엔트로피를 활용

In [45]:
import urllib.request

txt_filename = './data/2016-10-20.txt'

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename=txt_filename)

('./data/2016-10-20.txt', <http.client.HTTPMessage at 0x7fab5ef4b760>)

In [47]:
from soynlp import DoublespaceLineCorpus

# 말뭉치에 대해서 다수의 문서로 분리
corpus = DoublespaceLineCorpus(txt_filename)
len(corpus)

30091

In [48]:
# 공백이 아닌 문서에 한해 상위 3개의 문서만 출력

i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 3:
    break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

In [49]:
# 비지도학습이라 기존의 형태소 분석기들과는 달리 학습 과정을 거쳐야함
# 아래 코드는 전체 코퍼스로부터 응집확률과 브랜칭 엔트로피 단어 점수표를 만드는 과정
# WordExtractor()을 통해 전체 코퍼스에 대해 단어 점수표를 계산해야 함

from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 2.487 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


##### soynlp의 응집 확률

- 응집 확률 : 내부 문자열이 얼마나 응집하여 자주 등장하는지 판단하는 척도
- 문자열을 문자 단위로 분리하여 내부 문자열을 만드는 과정에서,
- 왼쪽부터 순서대로 문자를 추가하면서 각 문자열이 주어졌을 때 그다음 문자가 나올 확률을 계산하여 누적 곱을 한 값
- 값이 높을수록 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높음

In [50]:
# ex) '반포한강공원'의 응집 확률 계산

word_score_table["반포한"].cohesion_forward

0.08838002913645132

In [51]:
word_score_table["반포한강"].cohesion_forward

0.19841268168224552

In [52]:
word_score_table["반포한강공"].cohesion_forward

0.2972877884078849

In [53]:
word_score_table["반포한강공원"].cohesion_forward

0.37891487632839754

In [54]:
word_score_table["반포한강공원에"].cohesion_forward

0.33492963377557666

- 위 예시에서 응집 확률이 가장 높은 단어는 '반포한강공원'
- 즉, '반포한강공원'이 한 단어일 확률이 높다

#### soynlp의 브랜칭 엔트로피(branching entropy)

- 확률 분포의 엔트로피값
- = 주어진 문자열에서 다음 문자가 등장할 수 있는 가능성을 판단하는 척도
- 브랜칭 엔트로피의 값은 하나의 완성된 단어에 가까워질수록 문맥으로 인해 정확히 예측할 수 있게 되므로 점차 줄어듬
---

브랜칭 엔트로피 시뮬레이션 예시

<img src='./imgs/entropy.png' width = 550>

In [55]:
word_score_table["디스"].right_branching_entropy

1.6371694761537934

In [59]:
# 다음 값이 '레'가 나올게 너무 명확해서 0이 나옴

word_score_table["디스플"].right_branching_entropy

-0.0

In [57]:
# 다음 값이 '이'가 나올게 너무 명확해서 0이 나옴

word_score_table["디스플레"].right_branching_entropy

-0.0

In [58]:
# 뒤에 나올 단어가 무궁무진해서 엔트로피가 증가함

word_score_table["디스플레이"].right_branching_entropy

3.1400392861792916

#### soynlp의 LTokenizer

In [60]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [61]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

Q. DTM, TF-IDF, LSA, LDA에 대해서 설명해 볼까요?
 - DTM (Document-Term Matrix)은 문서-단어 행렬을 의미합니다. 각 문서에서 사용된 단어의 빈도를 표시하며, 행은 문서를 나타내고 열은 단어를 나타냅니다. 텍스트 데이터의 각 문서를 벡터로 표현할 수 있으며, 이를 활용해 문서 간 유사도를 계산하거나 토픽 모델링 등의 분석을 수행할 수 있습니다.

 - TF-IDF (Term Frequency-Inverse Document Frequency)는 DTM에서 특정 단어가 얼마나 중요한지를 나타내는 지표입니다. 많이 나오는 일반적인 단어들은 중요하지 않은 단어로 간주되고, 문서를 잘 나타내는 중요한 단어들이 높은 점수를 받게 됩니다.

 - LSA (Latent Semantic Analysis)는 DTM을 활용해 문서 간 유사도를 계산하는 방법 중 하나입니다.
 단어-문서 행렬에서 잠재적인 의미를 추출하여 문서를 벡터 공간 상에 표현합니다. 이를 통해 문서 간 유사도를 계산할 수 있습니다.

 - LDA (Latent Dirichlet Allocation)는 문서 내의 토픽 분포와 토픽 내 단어 분포를 추정하여 토픽 모델링을 수행하는 방법입니다. 각 문서는 여러 개의 토픽으로 구성되어 있으며, 각 토픽은 단어들의 확률 분포로 나타내어집니다. 이를 통해 각 문서가 어떤 주제를 다루는지를 파악할 수 있습니다.

Q. soynlp를 간단하게 설명해 볼까요?
- soynlp는 한국어 자연어 처리를 위한 파이썬 라이브러리입니다.
특히, 자주 등장하는 신조어나 오탈자, 그리고 형태소 분석에서 발생하는 OOV(Out-of-Vocabulary) 문제를 처리하는 데 강점을 가지고 있습니다. 데이터에서 단어 빈도수를 기반으로 단어들을 자동으로 추출하며, 이를 활용해 형태소 분석을 수행합니다.